In [ ]:
# default_exp files

In [ ]:
#export
import requests

from yx_motor.api import API

class Files:
    "Class for handling authenticate API actions"
    
    def __init__(self, api: API):
        self.api = api
        self.base_endpoint = 'files/'
        
    def download_file(self, file_uuid: str, download_path: str):
        #TODO: MVP
        pass
    
    def upload_file(self):
        #TODO: MVP
        pass
    
    def update_file(self):
        #TODO: MVP
        pass
    
    def get_file_versions(self):
        pass
    
    def delete_file(self, hard=False):
        pass
    
    def copy_file(self, source_path: str, target_path: str):
        pass
    
    def move_file(self, source_path: str, target_path: str):
        #TODO: MVP
        pass
    
    def restore_deleted_file(self, asset_path: str=None, asset_id: str=None):
        pass

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_client.ipynb.
Converted 02_jobs.ipynb.
Converted 03_api.ipynb.
Converted 04_authenticate.ipynb.
Converted 05_files.ipynb.
Converted index.ipynb.
